### Process stats for multiband image

In [2]:
%load_ext autoreload
%autoreload 2
    

In [23]:
import ee
import pandas as pd
import geemap
import geopandas as gpd
import modules.tidy_tables as tidy_tables
import json
import numpy as np
import time

from modules.gee_initialize import initialize_ee 

initialize_ee()

# from parameters.config_runtime import country_dataset_name
from parameters.config_runtime import (
    geo_id_column, 
    geometry_area_column, 
    plot_id_column, 
    lookup_gee_datasets_df,
    keep_system_index,
    keep_original_properties
    )

from modules.tidy_tables import create_column_list_from_lookup,order_list_from_lookup

In [4]:
# %run alt_whisp_prep_multiband_image.ipynb # get stats and image
from modules.stats import get_stats, get_stats_formatted 

##### Adding lists for reformatting manually here for now for faster processing (avoid use of the parameters/lookup_gee_datasets_df.csv)

In [5]:

# make lists for formatting - including a few distinctions for dataset types and preferred formatting
def make_processing_lists_from_gee_datasets_lookup(lookup_gee_datasets_df):
    # global all_dataset_list
    global exclude_list
    global all_datasets_list
    global presence_only_flag_list
    global decimal_place_column_list
    global order_list


    exclude_list = list(lookup_gee_datasets_df["dataset_name"][(lookup_gee_datasets_df["exclude"]==1)])

    # use the exclude list to filter lookup so all subsequent lists dont contain them
    lookup_gee_datasets_df = lookup_gee_datasets_df[(lookup_gee_datasets_df["exclude"]!=1)]
    
    all_datasets_list = list(lookup_gee_datasets_df["dataset_name"])
    
    presence_only_flag_list = list(lookup_gee_datasets_df["dataset_name"][(lookup_gee_datasets_df["presence_only_flag"]==1)])
          
    decimal_place_column_list =  [i for i in all_datasets_list if i not in presence_only_flag_list]

    order_list = order_list_from_lookup(lookup_gee_datasets_df) 
    
    return 
    exclude_list
    all_datasets_list,
    buffer_stats_list, 
    presence_only_flag_list, 
    decimal_place_column_list,
    order_list

make_processing_lists_from_gee_datasets_lookup(lookup_gee_datasets_df)

# buffer_stats_list 
# presence_only_flag_list
decimal_place_column_list
# exclude_list

from modules.datasets import combine_datasets

multiband_image_list = combine_datasets().bandNames()#.remove("Area_ha")

in_both_lists = multiband_image_list.filter(ee.Filter.inList('item', all_datasets_list));
not_in_multiband = multiband_image_list.filter(ee.Filter.inList('item', all_datasets_list).Not());
not_in_lookup  = ee.List(all_datasets_list).filter(ee.Filter.inList('item', multiband_image_list).Not());
# print ("number_in_multiband_datasets_list", multiband_image_list.length().getInfo())
# print ("number_in_both_lists",in_both_lists.length().getInfo())
# print ("exclude_list",exclude_list)
# print ("not_in_multiband",not_in_multiband.getInfo())

in_lookup = multiband_image_list.containsAll(ee.List(all_datasets_list)).getInfo()
print ("Datasets present in lookup: ", in_lookup)
if in_lookup ==False:
    print("missing from lookup",not_in_lookup.getInfo())

# multiband_datasets_list
# decimal_place_column_list

Datasets present in lookup:  True


In [19]:
id_name = "PlotID"

prefix_columns_list =[geo_id_column,geometry_area_column, "Country"] 
print(prefix_columns_list)


if geo_id_column in roi.first().propertyNames().getInfo():
    print ("geo id present")
else:
    prefix_columns_list.remove(geo_id_column)
    print ("geo id not present",prefix_columns_list) 


column_order_list = prefix_columns_list + order_list

# len (column_order_list)


['Geo_id', 'Area_ha', 'Country']
geo id not present ['Area_ha', 'Country']


['Area_ha', 'Country']

In [ ]:
st = time.time()

%store -r roi



## if over a certain number of polygons, then batch export to drive and use the formatting for that  else, do on the fly

## if going to drive uses a different cleaning process. Mostly sorts flagging and decimals.
    # i.e. doesnt do order currently as not possible in a fc, and wont remove underscores as getinfo is client side


## 
if ee.FeatureCollection(roi).size().getInfo()>500:#update this code to depend on a variable? or if memory error?
    print ("Over size limit for on the fly processing. \n Attempting export to Google Drive 'Whisp_results/Whisp_output_table_export.csv' as a batch task... ")
    
    stats_fc_formatted = get_stats(roi)
    
    stats_fc_formatted = get_stats_formatted(
    roi,
    # order = column_order_list, # not working as reverts to alphabetical
    id_name = id_name,
    flag_positive = presence_only_flag_list, #,
    # round_properties = decimal_place_column_list#,
    exclude_properties = exclude_list#,
    # select_and_rename = True # underscore removal (DONT RUN - needs debugging relies on get info currently, so only for small runs)
    )

    task = ee.batch.Export.table.toDrive(collection=stats_fc_formatted, 
                     description="whisp_output_table_export", 
                     folder="Whisp_results",
                     fileFormat="CSV")


    task.start()
    
    df_out =  ("Task running. Check https://code.earthengine.google.com/tasks. You can manually upload to Sepal when complete")
    
    print (df_out)
else:
    print ('...stats processing started')
    
    def add_system_index_as_property_to_feature(feature):
        # Get the system:index of the feature
        system_index = feature.get('system:index')
           
        # Set the 'id' property of the feature
        return feature.set('system_index',system_index)
    
    def add_system_index_as_property_to_fc(feature_col):    
       return feature_col.map(add_system_index_as_property_to_feature)

################
    
    stats_fc_formatted = get_stats_formatted(
    roi,
    # order = column_order_list, # not working as reverts to alphabetical
    id_name = id_name,
    flag_positive = presence_only_flag_list, #,
    # round_properties = decimal_place_column_list#,
    exclude_properties = exclude_list#,
    # select_and_rename = True # underscore removal (DONT RUN - needs debugging relies on get info currently, so only for small runs)
    )
    def add_new_elements(existing_list, new_elements):
        existing_list.extend([element for element in new_elements if element not in existing_list])

    # stats_fc_formatted = get_stats(roi)
    
    if keep_system_index:
        stats_fc_formatted = add_system_index_as_property_to_fc(stats_fc_formatted)
        add_new_elements(column_order_list,['system:index'])

    if keep_original_properties:
        original_columns = roi.first().propertyNames().getInfo()
        add_new_elements(column_order_list,original_columns)
    
    df = geemap.ee_to_df(stats_fc_formatted).rename(columns={'system_index':'system:index'})

    df = df.reindex(columns=column_order_list)  
    
    df_out = df  

    #####
    # stats_fc = get_stats(roi)
    # if keep_system_index:
    #     stats_fc = add_system_index_as_property_to_fc(stats_fc)

    # df = geemap.ee_to_df(stats_fc)  
  
    # df_out = df  
   
#    # df_out = process_dataframe(df, geo_id_column, geometry_area_column, country_dataset_name, lookup_gee_datasets_df,
#    #                       presence_only_flag_list, decimal_place_column_list, plot_id_column,keep_system_index=keep_system_index,debug=False)

    print ('...ee to df processing time:', time.time() - st, 'seconds')# get the execution time
    

In [ ]:

# # # Define a sample feature with properties
# # sample_feature = ee.Feature({
# #     'property1': 3.75,
# #     'property2': 7.2,
# #     'property3': None  # Add a property with None value for testing
# # })

# # # Define a list of properties to round
# # round_properties = ['property1', 'property2', 'property3']  # Adjust based on your actual properties

# # # Function to round properties to whole numbers
# # def round_properties_to_whole_numbers(feature, round_properties):
# #     for prop_name in round_properties:
# #         try:
# #             # Retrieve the property value
# #             prop_value = feature.get(prop_name)
            
# #             # Check if the property value is not None
# #             if prop_value is not None:
# #                 # Round the property value to the nearest whole number
# #                 prop_value_rounded = ee.Number(prop_value).round()
                
# #                 # Update the feature with the rounded property value
# #                 feature = feature.set(prop_name, prop_value_rounded)
# #         except Exception as e:
# #             # Print the error message
# #             print(e)
            
# #     return feature


# # Function to round properties to whole numbers
# def round_properties_to_whole_numbers(feature, round_properties):
                                      
#     property_list = feature.propertyNames()
                                      
#     round_properties_filtered = ee.List(round_properties).filter(ee.Filter.inList('item', property_list));    
#     round_properties_filtered = round_properties_filtered.getInfo()
#     for prop_name in round_properties_filtered:
#         try:
#             # Retrieve the property value
#             prop_value = feature.get(prop_name)
            
#             feature = feature.set(prop_name, ee.Number(prop_value).round())
            
#             # # Check if the property value is not None
#             # if prop_value is not None:
#             #     # Get the type of the property value
#             #     prop_type = ee.Algorithms.ObjectType(prop_value).getInfo()
                
#             #     # Ensure prop_value is a valid number before rounding
#             #     if prop_type in ['Integer', 'Double', 'Long']:
#             #         # Round the property value to the nearest whole number
#             #         prop_value_rounded = ee.Number(prop_value).round()
                    
#             #         # Update the feature with the rounded property value
#             #         feature = feature.set(prop_name, prop_value_rounded)
#             #     else:
#             #         print(f"Skipping property '{prop_name}' as it is not a valid number.")
#         except ee.EEException as e:
#             # Print the error message
#             print(f"Error processing property '{prop_name}': {e}")
#             # Skip to the next property
#             continue
#         except Exception as e:
#             # If it's not an EEException, print the error and re-raise the exception
#             print(f"Unexpected error processing property '{prop_name}': {e}")
#             raise
            
#     return feature



# # Test the function with the sample feature
# # rounded_feature = round_properties_to_whole_numbers(sample_feature, round_properties)

# # Print the result
# # print(rounded_feature.getInfo())
# # %store -r roi
# sample_feature = get_stats(roi.limit(2)).first()

# round_properties = (
# ['EUFO_2020',
#  'GLAD_Primary',
#  'TMF_undist',
#  'JAXA_FNF_2020',
#  'GFC_TC_2020',
#  'ESA_TC_2020',
#  'ESRI_TC_2020',
#  'TMF_disturbed',
#  'TMF_plant',
#  'Oil_palm_Descals',
#  'Oil_palm_FDaP',
#  'Cocoa_ETH',
#  'GFC_Loss_Year_2001',
#  'GFC_Loss_Year_2002',
#  'GFC_Loss_Year_2003',
#  'GFC_Loss_Year_2004',
#  'GFC_Loss_Year_2005',
#  'GFC_Loss_Year_2006',
#  'GFC_Loss_Year_2007',
#  'GFC_Loss_Year_2008',
#  'GFC_Loss_Year_2009',
#  'GFC_Loss_Year_2010',
#  'GFC_Loss_Year_2011',
#  'GFC_Loss_Year_2012',
#  'GFC_Loss_Year_2013',
#  'GFC_Loss_Year_2014',
#  'GFC_Loss_Year_2015',
#  'GFC_Loss_Year_2016',
#  'GFC_Loss_Year_2017',
#  'GFC_Loss_Year_2018',
#  'GFC_Loss_Year_2019',
#  'GFC_Loss_Year_2020',
#  'GFC_Loss_Year_2021',
#  'GFC_Loss_Year_2022'
# ])

# rounded_feature = round_properties_to_whole_numbers(sample_feature, round_properties)
# rounded_feature
# # sample_feature


# # df = geemap.ee_to_df(ee.FeatureCollection([rounded_feature]))  